<a href="https://colab.research.google.com/github/rahatarinasir/Dayche/blob/master/auto_encoder_rough.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
## Initialization

In [ ]:
import keras 
import tensorflow as tf
from keras.datasets import mnist
import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from keras.layers import Layer
from keras import backend as K
from tensorflow.keras.layers import Input
from keras.models import Functional
from keras.layers import Dense
from sklearn.metrics import confusion_matrix, classification_report


In [ ]:
## Data loading

In [ ]:
data = load_breast_cancer()
X = data.data
y = data.target
#label = np.zeros((len(y), 2))
#label[np.where(y == 1), 1] = 1
#label[np.where(y == 0), 0] = 1
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3)
scaler = MinMaxScaler().fit(X_train)
X_train, X_test = scaler.transform(X_train), scaler.transform(X_test)



In [ ]:
## Define Rough Layer

In [ ]:
class RoghLayer(Layer):
    def __init__(self, units, alpha = 0.5, beta = 0.5, activation = None, **kwargs):
        super(RoghLayer, self).__init__(**kwargs)
        self.units = units
        self.activation = activation
        self.alpha = alpha
        self.beta = beta
        assert (self.alpha + self.beta == 1)

    def build(self, input_shape):
        self.wl = self.add_weight(name='lower_weights',
                                  shape=(int(input_shape[1]), self.units),
                                  initializer='uniform',
                                  trainable=True)
        self.bl = self.add_weight(name='lower_bias',
                                  shape=(1, self.units),
                                  initializer='uniform',
                                  trainable=True)
        self.wu = self.add_weight(name='upper_weights',
                                  shape=(int(input_shape[1]), self.units),
                                  initializer='uniform',
                                  trainable=True)
        self.bu = self.add_weight(name='upper_bias',
                                  shape=(1, self.units),
                                  initializer='uniform',
                                  trainable=True)
            
        super(RoghLayer, self).build(input_shape)

    def call(self, inputs):
        net_l = tf.matmul(inputs, self.wl) + self.bl
        net_u = tf.matmul(inputs, self.wu) + self.bu
        if self.activation == None:
            ol = netl
            ou = netu
        elif self.activation == 'sigmoid':
            ol = tf.keras.activations.sigmoid(net_l)
            ou = tf.keras.activations.sigmoid(net_u)
        elif self.activation == 'tanh':
            ol = tf.keras.activations.tanh(net_l)
            ou = tf.keras.activations.tanh(net_u)
        elif self.activation == 'relu':
            ol = tf.keras.activations.relu(net_l)
            ou = tf.keras.activations.relu(net_u)
    
        o = self.alpha * ol + self.beta * ou
        return o


In [ ]:
## Build a Rough autoencoder

In [ ]:
###from customized_layers import RoughLayer

import os
print(os.getcwd())


In [ ]:
input_layer = tf.keras.Input(shape=(30,))
encoder =RoghLayer(2)
decoder=Dense(30, activation='linear')
autoencoder = tf.keras.Model(encoder, decoder)
decoder_layer = autoencoder.layers[-1]
encoded_input = tf.keras.Input(shape=(2,))

ValueError: Input tensors to a Functional must come from `tf.keras.Input`. Received: <__main__.RoghLayer object at 0x0000027A82BC3E80> (missing previous layer metadata).

In [ ]:

input_layer = keras.Input(shape=(30,))

encoded = tf.keras.layers.add(RoghLayer(2, activation = 'sigmoid', name = 'Rough'))(input_img)
decoded = keras.layers.Dense(30, activation='linear')(encoded)
autoencoder = keras.Model(input_img, decoded)
encoder = keras.Model(input_img, encoded)
decoder_layer = autoencoder.layers[-1]
encoded_input = keras.Input(shape=(2,))
decoder = keras.Model(encoded_input, decoder_layer(encoded_input))



In [ ]:
autoencoder.compile(optimizer='adam', loss='mse', metrics = 'mse')
autoencoder.fit(X_train, X_train,
                epochs=100,
                batch_size=5,
                shuffle=True,
                validation_data=(X_test, X_test))

In [ ]:
encoded_x_test = encoder(X_test)
encoded_x_train = encoder(X_train)

In [ ]:
classifier = keras.models.Sequential()
classifier.add(keras.layers.Dense(5, activation = 'relu'))
classifier.add(keras.layers.Dense(1, activation = 'sigmoid'))
classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = 'accuracy')
classifier.fit(encoded_x_train, y_train, validation_data = (encoded_x_test, y_test), epochs = 100, batch_size = 5 )

In [ ]:
y_pre = classifier.predict(encoded_x_test)
y_pre[y_pre >= 0.5] = 1
y_pre[y_pre < 0.5] = 0
print(confusion_matrix(y_pre, y_test))
print(classification_report(y_pre, y_test))